# **Final Data Processing and combining lipinski and molecular descriptors**

In this part we combine the lipinski and molecular descriptor dataframes.
This is done to improve the overall score of the model.
We also employ the **IWSSR** (*Incremental Weighted Support Sampled Regression*) algorithm to select the best features affecting the pIC50 value out of the 881 molecule descriptors.
While the effects of quick computation are as prominent for a small dataset like ours (86 vectors), the technique can do very well in reducing bias and make the training of the model difficult. This is because IWSSR reduces duplicated, irrelevant and Noisy features.

Only the best features out of the 881 will be selected in an attempt to improve the r2 score.

We will also scale the relevant features since the lipinski descritors (MW, HBondDonors, HBondAcceptors) are of different units as compared to the molecular descriptors obtained from the PaDEL library.
We'll be using the RotationForest model, this requires all features including the output to be scaled.

Finally we split the data into training and test subsets to calculate our score.

---


In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy

In [2]:
lipinskiDes = pd.read_csv('A8.coronavirus_LipinskiDescriptors.csv')
selection = ['MW','NumHAcceptors','NumHDonors']
lipinskiDes_select = lipinskiDes[selection]
molDes = pd.read_csv('A7.coronavirus_MolecularDescriptors_pubchem.csv')
pic50 = pd.read_csv('A9.coronavirus_pIC50.csv')

In [3]:
lipinskiDes_select

,MW,NumHAcceptors,NumHDonors
0,281.271,5.0,0.0
1,415.589,2.0,0.0
2,421.190,4.0,0.0
3,293.347,3.0,0.0
4,338.344,5.0,0.0
...,...,...,...
81,338.359,5.0,0.0
82,296.366,3.0,0.0
83,276.291,3.0,0.0
84,278.307,3.0,0.0


In [4]:
molDes

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
82,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
83,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
84,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
pic50

,pIC50
0,5.142668
1,5.026872
2,4.869666
3,4.882397
4,5.698970
...,...
81,4.675718
82,3.644548
83,4.412289
84,4.841638


# Applying IWSSR to the molecular descriptors dataset, restricting to the best n_feat features

In [20]:
n_feat = 468

In [7]:
def symmetric_uncertainty(df, feature, target):
    """
    Calculate the Symmetric Uncertainty (SU) between a feature and the target variable.

    Parameters:
    df (pd.DataFrame): The dataframe containing the dataset.
    feature (str): The name of the feature column.
    target (str): The name of the target column.

    Returns:
    float: The Symmetric Uncertainty value.
    """

    # Calculate the entropy of the feature and target
    H_feature = entropy(df[feature].value_counts(normalize=True), base=2)
    H_target = entropy(df[target].value_counts(normalize=True), base=2)

    # Calculate the mutual information between the feature and the target
    I_feature_target = mutual_info_score(df[feature], df[target])

    # Calculate the Symmetric Uncertainty
    SU = 2 * I_feature_target / (H_feature + H_target)

    return SU

In [8]:
def symmetric_uncertainty_df(X_train, Y_train):
    """
    Calculate the Symmetric Uncertainty (SU) for all features in X_train with respect to Y_train.

    Parameters:
    X_train (pd.DataFrame): The dataframe containing the input features.
    Y_train (pd.Series): The series containing the target variable.

    Returns:
    dict: A dictionary where keys are feature names and values are their SU values.
    """
    su_values = {}
    df = X_train.copy()
    df['target'] = Y_train

    for feature in X_train.columns:
        su_values[feature] = symmetric_uncertainty(df, feature, 'target')

    return su_values

In [9]:
SU_values = symmetric_uncertainty_df(molDes, pic50['pIC50'])
SU_values

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.1

{'PubchemFP0': 0.0,
 'PubchemFP1': 0.08945242792748034,
 'PubchemFP2': 0.1530581132370148,
 'PubchemFP3': 0.04945426637533352,
 'PubchemFP4': 0.0,
 'PubchemFP5': 0.0,
 'PubchemFP6': 0.0,
 'PubchemFP7': 0.0,
 'PubchemFP8': 0.0,
 'PubchemFP9': 0.0,
 'PubchemFP10': 0.0,
 'PubchemFP11': 0.0,
 'PubchemFP12': 0.17527129393699603,
 'PubchemFP13': 0.0,
 'PubchemFP14': 0.10193023497019452,
 'PubchemFP15': 0.16843781980815337,
 'PubchemFP16': 0.14353949633585478,
 'PubchemFP17': 0.0,
 'PubchemFP18': 0.0,
 'PubchemFP19': 0.08034965759396288,
 'PubchemFP20': 0.14353949633585475,
 'PubchemFP21': 0.021142847535204665,
 'PubchemFP22': 0.0,
 'PubchemFP23': 0.08950985028277875,
 'PubchemFP24': 0.07567891665815892,
 'PubchemFP25': 0.021142847535204665,
 'PubchemFP26': 0.0,
 'PubchemFP27': 0.0,
 'PubchemFP28': 0.0,
 'PubchemFP29': 0.0,
 'PubchemFP30': 0.0,
 'PubchemFP31': 0.0,
 'PubchemFP32': 0.0,
 'PubchemFP33': 0.17586957267123876,
 'PubchemFP34': 0.09242173276660148,
 'PubchemFP35': 0.0441863395211039

In [22]:
threshold = pd.Series(SU_values).nlargest(n_feat)
print(threshold)
# You can increase the value of n_feat to see that after 468 features the SU score is 0
# This means that by selecting all 880 molecular descriptors we were adding a lot of noise to the system
t_value = threshold[-1]
print(t_value)

PubchemFP181    0.186783
PubchemFP180    0.186580
PubchemFP593    0.185273
PubchemFP601    0.184240
PubchemFP375    0.183064
                  ...   
PubchemFP505    0.011434
PubchemFP583    0.011434
PubchemFP610    0.011434
PubchemFP764    0.011434
PubchemFP827    0.011434
Length: 468, dtype: float64
0.011433834581007916


In [23]:
for feature in molDes.columns:
    if SU_values[feature] < t_value:
        molDes.drop(feature, axis=1, inplace=True)
molDes

,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP12,PubchemFP14,PubchemFP15,PubchemFP16,PubchemFP19,PubchemFP20,PubchemFP21,...,PubchemFP822,PubchemFP824,PubchemFP826,PubchemFP827,PubchemFP828,PubchemFP830,PubchemFP831,PubchemFP833,PubchemFP834,PubchemFP835
0,1,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1,0,0,1,1,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,1,1,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
82,1,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
83,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
84,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Combining the requried dataframes

In [24]:
df = pd.concat([lipinskiDes_select, molDes, pic50],axis=1)
df.head()

,MW,NumHAcceptors,NumHDonors,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP12,PubchemFP14,PubchemFP15,PubchemFP16,...,PubchemFP824,PubchemFP826,PubchemFP827,PubchemFP828,PubchemFP830,PubchemFP831,PubchemFP833,PubchemFP834,PubchemFP835,pIC50
0,281.271,5.0,0.0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,5.142668
1,415.589,2.0,0.0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,5.026872
2,421.190,4.0,0.0,1,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,4.869666
3,293.347,3.0,0.0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,4.882397
4,338.344,5.0,0.0,1,0,0,1,1,1,0,...,0,0,0,0,0,0,1,0,0,5.698970


# Scaling the values

In [26]:
scaler = StandardScaler()
array = df.to_numpy()
scaled = scaler.fit_transform(array)
df = pd.DataFrame(scaled, columns=df.columns)
print(df.shape)
df.head()

(86, 472)


,MW,NumHAcceptors,NumHDonors,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP12,PubchemFP14,PubchemFP15,PubchemFP16,...,PubchemFP824,PubchemFP826,PubchemFP827,PubchemFP828,PubchemFP830,PubchemFP831,PubchemFP833,PubchemFP834,PubchemFP835,pIC50
0,-1.046883,-0.111915,-0.983696,0.29767,-0.694808,-0.190117,-1.267304,0.362738,0.713283,-0.658281,...,-0.154303,-0.108465,-0.108465,-0.108465,-0.154303,-0.108465,-0.108465,-0.108465,-0.108465,0.304375
1,0.440308,-1.555624,-0.983696,0.29767,-0.694808,-0.190117,-1.267304,0.362738,-1.401969,-0.658281,...,-0.154303,-0.108465,-0.108465,-0.108465,-0.154303,-0.108465,-0.108465,-0.108465,-0.108465,0.184652
2,0.502323,-0.593151,-0.983696,0.29767,-0.694808,-0.190117,0.789076,0.362738,-1.401969,-0.658281,...,6.480741,-0.108465,-0.108465,-0.108465,-0.154303,-0.108465,-0.108465,-0.108465,-0.108465,0.022115
3,-0.913175,-1.074388,-0.983696,0.29767,-0.694808,-0.190117,0.789076,0.362738,-1.401969,-0.658281,...,-0.154303,-0.108465,-0.108465,-0.108465,-0.154303,-0.108465,-0.108465,-0.108465,-0.108465,0.035278
4,-0.414961,-0.111915,-0.983696,0.29767,-0.694808,-0.190117,0.789076,0.362738,0.713283,-0.658281,...,-0.154303,-0.108465,-0.108465,-0.108465,-0.154303,-0.108465,9.219544,-0.108465,-0.108465,0.879543


In [28]:
df.to_csv('A10.coronavirus_Descriptor_IWSSR_Scaled.csv')
!zip -r coronavirus_4 A10.coronavirus_Descriptor_IWSSR_Scaled.csv

  adding: A10.coronavirus_Descriptor_IWSSR_Scaled.csv (deflated 96%)


# Data processing is complete!
We have 472 features left.Next step is to perform the train/test split and fit models.